## Problem 1

(a)
	There are 807 songs and 2421 users in the training dataset.
    Mean rating:  1.322631184306889
    Median rating:  1.0 
    Standard deviation of rating:  0.46088622855102235
    Minimum rating:  1.0 
    Maximum rating:  4.768655987487031 

(b)
    (i)
        Total parameters: 807 + 2421 = 3228
		Total observations: 807 * 2421 = 243,103
    (ii)
        The three most popular songs can be obtained by finding the songs with the largest absolute β coefficients.
           Rank  SongID      SongName            ArtistName
            1     54      You're The One        Dwight Yoakam
            2     26          Undo                  Bjork
            3     439        Secrets            One Republic
    (iii)
        The three most enthusiastic users can be obtained by finding the users with the largest absolute α coefficients.
               Rank  usersID     
                1     1540    
                2     838      
                3     1569    
    (iv)
            MAE: 
            RMSE: 
            OSR2: 

(c)
    (i)
        N = (2421 + 807)*k + 2421 + 807
          = 3228*(k+1)
        We will have 243,103 observations on training set.
    (ii)
        With the plot, we are going to find the number of archetypes that minimum MAE.
        Thus, we choose k = 6
    (iii)     
            MAE: 
            RMSE: 
            OSR2: 

(d)
    (i)
        In this part, I fitted Linear Regression and Random Forest with 'genre' and 'song'.
                         Linear Regression        Random Forest
            MAE 
            RMSE 
            OSR2 
    (ii)
                        Linear Regression        Random Forest            CF              Blended
            MAE 
            RMSE 
            OSR2 

In [1]:
import numpy as np
import pandas as pd

In [2]:
MusicRatingsTrain = pd.read_csv("MusicRatingsTrain.csv", index_col = 0)
MusicRatingsValidationA = pd.read_csv("MusicRatingsValidationA.csv", index_col = 0)
MusicRatingsValidationB = pd.read_csv("MusicRatingsValidationB.csv", index_col = 0)
MusicRatingsTest = pd.read_csv("MusicRatingsTest.csv", index_col = 0)

In [3]:
MusicRatingsTrain = pd.get_dummies(MusicRatingsTrain, columns=['genre'])
MusicRatingsValidationA = pd.get_dummies(MusicRatingsValidationA, columns=['genre'])
MusicRatingsValidationB = pd.get_dummies(MusicRatingsValidationB, columns=['genre'])
MusicRatingsTest = pd.get_dummies(MusicRatingsTest, columns=['genre'])

In [4]:
MusicRatingsTrain.sort_index()
MusicRatingsTrain_df = MusicRatingsTrain.pivot_table(index="userID", columns = "songID", values = "rating", dropna=False)
MusicRatingsTrain_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 1 to 2421
Columns: 807 entries, 1 to 807
dtypes: float64(807)
memory usage: 14.9 MB


In [5]:
# Compute the summary statistics for the rating variable
mean_rating = MusicRatingsTrain["rating"].mean()
median_rating = MusicRatingsTrain["rating"].median()
std_rating = MusicRatingsTrain["rating"].std()
min_rating = MusicRatingsTrain["rating"].min()
max_rating = MusicRatingsTrain["rating"].max()

# Print the results
print("Mean rating: ", mean_rating)
print("Median rating: ", median_rating)
print("Standard deviation of rating: ", std_rating)
print("Minimum rating: ", min_rating)
print("Maximum rating: ", max_rating)

Mean rating:  1.3226311843072156
Median rating:  1.0
Standard deviation of rating:  0.46088622855102235
Minimum rating:  1.0
Maximum rating:  4.768655987487031


In [6]:
MusicRatingsValidationA.sort_index()
MusicRatingsValidationA_df = MusicRatingsValidationA.pivot_table(index="userID", columns = "songID", values = "rating", dropna=False)
#movie_lens_valA_df.info()
valA_extra_columns = list(set(range(1,807+1)) - set(MusicRatingsValidationA_df.columns))
# print('# of missing columns: %s; # of missing musics: %s' % (2421-MusicRatingsValidationA_df.shape[1], len(valA_extra_columns)))
MusicRatingsValidationA_df[valA_extra_columns]=np.nan
MusicRatingsValidationA_df=MusicRatingsValidationA_df[MusicRatingsTrain_df.columns]
MusicRatingsValidationA_df = MusicRatingsTrain_df[[]].join(MusicRatingsValidationA_df, how='outer')
print(MusicRatingsValidationA_df.shape)

(2421, 807)


In [7]:
MusicRatingsValidationB.sort_index()
MusicRatingsValidationB_df = MusicRatingsValidationB.pivot_table(index="userID", columns = "songID", values = "rating", dropna=False)
valB_extra_columns = list(set(range(1,807+1)) - set(MusicRatingsValidationB_df.columns))
MusicRatingsValidationB_df[valB_extra_columns]=np.nan
MusicRatingsValidationB_df = MusicRatingsValidationB_df[MusicRatingsTrain_df.columns]
MusicRatingsValidationB_df = MusicRatingsTrain_df[[]].join(MusicRatingsValidationB_df, how='outer')
print(MusicRatingsValidationB_df.shape)

(2421, 807)


In [8]:
MusicRatingsTest.sort_index()
MusicRatingsTest_df = MusicRatingsTest.pivot_table(index="userID", columns = "songID", values = "rating", dropna=False)
test_extra_columns = list(set(range(1,3706+1)) - set(MusicRatingsTest_df.columns))
MusicRatingsTest_df[test_extra_columns]=np.nan
MusicRatingsTest_df = MusicRatingsTest_df[MusicRatingsTrain_df.columns]
MusicRatingsTest_df = MusicRatingsTrain_df[[]].join(MusicRatingsTest_df, how='outer')
print(MusicRatingsTest_df.shape)

(2421, 807)


In [9]:
MusicRatingsTrain_mat = MusicRatingsTrain_df.to_numpy()
MusicRatingsValidationA_mat = MusicRatingsValidationA_df.to_numpy()
MusicRatingsValidationB_mat = MusicRatingsValidationB_df.to_numpy()
MusicRatingsTest_mat = MusicRatingsTest_df.to_numpy()

In [10]:
MusicRatingsValidationA_mask = ~np.isnan(MusicRatingsValidationA_mat)
print(np.sum(MusicRatingsValidationA_mask))
MusicRatingsValidationB_mask = ~np.isnan(MusicRatingsValidationB_mat)
print(np.sum(MusicRatingsValidationB_mask))
MusicRatingsTest_mask = ~np.isnan(MusicRatingsTest_mat)
print(np.sum(MusicRatingsTest_mask))

14470
14471
14471


## Basic Analysis (without using features)

In [11]:
from fancyimpute import BiScaler
from fancyimpute import SoftImpute
MusicRatings_biscaler = BiScaler(scale_rows=False, scale_columns=False, verbose=True)
MusicRatingsTrain_mat_centered = MusicRatings_biscaler.fit_transform(MusicRatingsTrain_mat)
# MusicRatings_soft_imputer = SoftImpute(max_rank=5, verbose=True)
# MusicRatings_centered_filled = MusicRatings_soft_imputer.fit_transform(MusicRatingsTrain_mat_centered)

[BiScaler] Initial log residual value = 8.636690
[BiScaler] Iter 1: log residual = -1.283248, log improvement ratio=9.919937
[BiScaler] Iter 2: log residual = -2.468379, log improvement ratio=1.185132
[BiScaler] Iter 3: log residual = -3.602839, log improvement ratio=1.134459
[BiScaler] Iter 4: log residual = -4.731547, log improvement ratio=1.128708
[BiScaler] Iter 5: log residual = -5.851961, log improvement ratio=1.120415
[BiScaler] Iter 6: log residual = -6.959236, log improvement ratio=1.107275
[BiScaler] Iter 7: log residual = -8.045993, log improvement ratio=1.086756
[BiScaler] Iter 8: log residual = -9.101878, log improvement ratio=1.055885
[BiScaler] Iter 9: log residual = -10.114070, log improvement ratio=1.012192
[BiScaler] Iter 10: log residual = -11.069605, log improvement ratio=0.955535
[BiScaler] Iter 11: log residual = -11.959489, log improvement ratio=0.889884
[BiScaler] Iter 12: log residual = -12.782528, log improvement ratio=0.823039
[BiScaler] Iter 13: log residual

In [12]:

alpha = MusicRatings_biscaler.row_means
beta = MusicRatings_biscaler.column_means

user = pd.DataFrame(columns=['alpha'])
songs = pd.DataFrame(columns=['beta'])

# Sort users and songs by their alpha and beta values respectively
user['alpha'] = alpha
songs['beta'] = beta
users = user.sort_values('alpha', ascending=False)
songs = songs.sort_values('beta', ascending=False)

# index need to +1
print(users.head(3))
print(songs.head(3))

         alpha
1539  0.980499
837   0.822825
1568  0.810956
         beta
53   2.176931
25   2.146214
438  2.055549


In [13]:
MusicRatingsTrain[MusicRatingsTrain['songID']==54]

,songID,rating,songName,year,artist,genre_Country,genre_Electronic,genre_Folk,genre_Pop,genre_Rap,genre_RnB,genre_Rock
userID,,,,,,,,,,,,
685,54,1.000000,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
1515,54,2.484754,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
1291,54,2.149164,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
570,54,1.000000,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
274,54,3.173836,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1613,54,1.494918,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
2278,54,2.279344,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0
1738,54,3.517876,You're The One,1990,Dwight Yoakam,1,0,0,0,0,0,0


In [14]:
MusicRatingsTrain[MusicRatingsTrain['songID']==26]

,songID,rating,songName,year,artist,genre_Country,genre_Electronic,genre_Folk,genre_Pop,genre_Rap,genre_RnB,genre_Rock
userID,,,,,,,,,,,,
440,26,3.326332,Undo,2001,Bjork,0,0,0,0,0,0,1
2087,26,1.000000,Undo,2001,Bjork,0,0,0,0,0,0,1
357,26,3.538574,Undo,2001,Bjork,0,0,0,0,0,0,1
1916,26,2.149164,Undo,2001,Bjork,0,0,0,0,0,0,1
1592,26,2.568853,Undo,2001,Bjork,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
659,26,2.149164,Undo,2001,Bjork,0,0,0,0,0,0,1
1491,26,2.712135,Undo,2001,Bjork,0,0,0,0,0,0,1
330,26,1.784426,Undo,2001,Bjork,0,0,0,0,0,0,1


In [15]:
MusicRatingsTrain[MusicRatingsTrain['songID']==439]

,songID,rating,songName,year,artist,genre_Country,genre_Electronic,genre_Folk,genre_Pop,genre_Rap,genre_RnB,genre_Rock
userID,,,,,,,,,,,,
1844,439,1.494918,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
998,439,3.173836,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
2337,439,2.279344,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
878,439,1.494918,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
235,439,2.484754,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1900,439,2.149164,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
1330,439,2.279344,Secrets,2009,OneRepublic,0,0,0,0,0,0,1
491,439,2.774262,Secrets,2009,OneRepublic,0,0,0,0,0,0,1


In [16]:
MusicRatings_filled = MusicRatings_biscaler.inverse_transform(MusicRatingsTrain_mat_centered)
MusicRatings_filled = np.clip(MusicRatings_filled, 1, 5)
# MusicRatings_filled

In [17]:
def masked_mae(X_true, X_pred, mask):
    masked_diff = X_true[mask] - X_pred[mask]
    return np.mean(np.abs(masked_diff))

def masked_mse(X_true, X_pred, mask):
    masked_diff = X_true[mask] - X_pred[mask]
    return np.mean(masked_diff ** 2)

def OSR2(mse_model, mse_baseline):
    return 1 - mse_model/mse_baseline

In [ ]:
MusicRatingsTest_mae = masked_mae(MusicRatingsTest_mat, MusicRatings_filled, MusicRatingsTest_mask)

print("MAE %s " % (MusicRatingsTest_mae/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

MusicRatingsTest_mse = masked_mse(MusicRatingsTest_mat, MusicRatings_filled, MusicRatingsTest_mask)

print("RMSE %s " % (np.sqrt(MusicRatingsTest_mse)/4))

baseline_pred = np.mean(MusicRatingsTrain)[2]
baseline_model = baseline_pred*np.ones((2421, 807))
baseline_mse = masked_mse(MusicRatingsTest_mat, baseline_model, MusicRatingsTest_mask)

print("OSR2 %s " % OSR2(MusicRatingsTest_mse, baseline_mse))

MAE nan 
RMSE nan 


## Cross Validation

In [ ]:
MusicRatingsValidationA_mae_lst = []
for i in range(20):
    param_cv = i+1
    MusicRatings_soft_imputer_cv = SoftImpute(max_rank=param_cv, verbose=False)
    MusicRatings_centered_filled_cv = MusicRatings_soft_imputer_cv.fit_transform(MusicRatingsTrain_mat_centered)
    MusicRatings_filled_cv = MusicRatings_biscaler.inverse_transform(MusicRatings_centered_filled_cv)
    MusicRatings_filled_cv = np.clip(MusicRatings_filled_cv, 1, 5)
    MusicRatingsValidationA_mae_cv = masked_mae(MusicRatingsValidationA_mat, MusicRatings_filled_cv, MusicRatingsValidationA_mask)
    MusicRatingsValidationA_mae_lst.append(MusicRatingsValidationA_mae_cv)
    print('iter %s - Validation MAE %s' % (param_cv, MusicRatingsValidationA_mae_cv))

In [ ]:
# MusicRatingsTrain_mat_centered

In [ ]:
import matplotlib.pyplot as plt
x = range(1,21)
y = MusicRatingsValidationA_mae_lst
plt.figure(figsize=(4, 4))
plt.scatter(x, y, linewidth=2, color='black')
plt.xlabel('Number of Archetypal Users', fontsize=16)
plt.ylabel('Validation MAE', fontsize=16)
plt.show()

## Model Blending/Stacking

Collaborative Filtering

In [ ]:
MusicRatings_soft_imputer = SoftImpute(max_rank=10, verbose=False) #use the best 
MusicRatings_centered_filled = MusicRatings_soft_imputer.fit_transform(MusicRatingsTrain_mat_centered)
MusicRatings_filled_matrix = MusicRatings_biscaler.inverse_transform(MusicRatings_centered_filled)
MusicRatings_filled_matrix = np.clip(MusicRatings_filled_matrix, 1, 5)

In [ ]:
val_mae_cf = masked_mae(MusicRatingsValidationB_mat, MusicRatings_filled_matrix, MusicRatingsValidationB_mask)
print("MAE %s " % (val_mae_cf/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

val_mse_cf = masked_mse(MusicRatingsValidationB_mat, MusicRatings_filled_matrix, MusicRatingsValidationB_mask)
print("RMSE %s " % (np.sqrt(val_mse_cf)/4))

baseline_pred = np.mean(MusicRatingsTrain_df)[2]
baseline_model = baseline_pred*np.ones((2421, 807))
baseline_mse = masked_mse(MusicRatingsValidationB_mat, baseline_model, MusicRatingsValidationB_mask)

print("OSR2 %s " % OSR2(val_mse_cf, baseline_mse))

In [ ]:
val_pred_cf = MusicRatings_filled_matrix[MusicRatingsValidationB_mask]

blend_valB_df = MusicRatingsValidationB.reset_index()[['userID','songID','rating']]

blend_valB_df['val_pred_cf']=val_pred_cf
# blend_valB_df

Linear Regression

In [ ]:
# Some extra data preprocessing, converting numbers to categorical variables
MusicRatingsTrain['year'] = MusicRatingsTrain.year.astype('category')
MusicRatingsValidationA['year'] = MusicRatingsValidationA.year.astype('category')
MusicRatingsTest['year'] = MusicRatingsTest.year.astype('category')
# MusicRatingsTest

In [ ]:
# Some extra data preprocessing, converting numbers to categorical variables
MusicRatingsTrain = MusicRatingsTrain.drop(['songName','artist'], axis=1)
MusicRatingsValidationA = MusicRatingsValidationA.drop(['songName', 'artist'], axis=1)
MusicRatingsValidationB = MusicRatingsValidationB.drop(['songName', 'artist'], axis=1)
MusicRatingsTest = MusicRatingsTest.drop(['songName', 'artist'], axis=1)
MusicRatingsValidationB

In [ ]:
depedent_var = MusicRatingsTrain.columns.difference(["rating","userID","songID"])
print(len(depedent_var))
depedent_var = "+".join(depedent_var)
depedent_var

In [ ]:
import statsmodels.formula.api as smf
my_formula = "rating ~" + depedent_var
MusicRatings_ols = smf.ols(formula=my_formula, data=MusicRatingsTrain)
res = MusicRatings_ols.fit()
print(res.summary())

In [ ]:
MusicRatingsValidationB.reset_index(drop=True, inplace=True)
val_pred_ols = res.predict(MusicRatingsValidationB)
blend_valB_df['val_pred_ols']=val_pred_ols
blend_valB_df

In [ ]:
val_mae_ols = np.mean(np.abs(MusicRatingsValidationB_mat[MusicRatingsValidationB_mask] - val_pred_ols))
print("MAE %s " % (val_mae_ols/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

val_mse_ols = np.mean((MusicRatingsValidationB_mat[MusicRatingsValidationB_mask] - val_pred_ols)**2)
print("RMSE %s " % (np.sqrt(val_mse_ols)/4))

print("OSR2 %s " % OSR2(val_mse_ols, baseline_mse))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# MusicRatingsTrain_rest_index = MusicRatingsTrain.reset_index()
X_train = MusicRatingsTrain.drop(columns = ["songID","rating"])
y_train = MusicRatingsTrain["rating"]
rf = RandomForestRegressor(max_features=5, min_samples_leaf=5, 
                           n_estimators = 500, random_state=88, verbose=2)
rf.fit(X_train, y_train)

In [ ]:
val_pred_rf=rf.predict(MusicRatingsValidationB.drop(columns = ["songID","rating"]))

blend_valB_df['val_pred_rf']=val_pred_rf
blend_valB_df

In [ ]:
val_mae_rf = np.mean(np.abs(MusicRatingsValidationB_mat[MusicRatingsValidationB_mask] - val_pred_rf))
print("MAE %s " % (val_mae_rf/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

val_mse_rf = np.mean((MusicRatingsValidationB_mat[MusicRatingsValidationB_mask] - val_pred_rf)**2)
print("RMSE %s " % (np.sqrt(val_mse_rf)/4))

print("OSR2 %s " % OSR2(val_mse_rf, baseline_mse))

Combine three models using OLS

In [ ]:
blending_ols = smf.ols(formula='rating ~ val_pred_cf+val_pred_ols+val_pred_rf -1', data=blend_valB_df)
blending_res = blending_ols.fit()
print(blending_res.summary())

In [ ]:
val_pred_blended =blending_res.predict(blend_valB_df)
blend_valB_df['pred_blended']= val_pred_blended

blend_valB_df

In [ ]:
val_mae_blended = np.mean(np.abs(MusicRatingsValidationB_mat[MusicRatingsValidationB_mask] - val_pred_blended))
print("MAE %s " % (val_mae_blended/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

val_mse_blended = np.mean((MusicRatingsValidationB_mat[MusicRatingsValidationB_mask] - val_pred_blended)**2)
print("RMSE %s " % (np.sqrt(val_mse_blended)/4))

print("OSR2 %s " % OSR2(val_mse_blended, baseline_mse))

Test set performance

In [ ]:
baseline_pred = np.mean(MusicRatingsTrain_df)[2]
baseline_model = baseline_pred*np.ones((2421, 807))
baseline_mse = masked_mse(MusicRatingsTest_mat, baseline_model, MusicRatingsTest_mask)
baseline_mse

In [ ]:
test_mae_cf = masked_mae(MusicRatingsTest_mat, MusicRatings_filled_matrix, MusicRatingsTest_mask)
print("Test_cf MAE  %s " % (val_mae_cf/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

test_mse_cf = masked_mse(MusicRatingsTest_mat, MusicRatings_filled_matrix, MusicRatingsTest_mask)
print("Test_cf RMSE %s " % (np.sqrt(val_mse_cf)/4))

baseline_pred = np.mean(MusicRatingsTrain_df)[2]
baseline_model = baseline_pred*np.ones((2421, 807))
baseline_mse = masked_mse(MusicRatingsTest_mat, baseline_model, MusicRatingsTest_mask)

print("Test_cf OSR2 %s " % OSR2(val_mse_cf, baseline_mse))

In [ ]:
test_pred_cf = MusicRatings_filled_matrix[MusicRatingsTest_mask]

blend_test_df = MusicRatingsTest.reset_index()[['userID','songID','rating']]

blend_test_df['test_pred_cf']=test_pred_cf
blend_test_df

In [ ]:
MusicRatingsTest.reset_index(drop=True, inplace=True)
test_pred_ols = res.predict(MusicRatingsTest)
blend_test_df['test_pred_ols']=test_pred_ols
blend_test_df

In [ ]:
test_mae_ols = np.mean(np.abs(MusicRatingsTest_mat[MusicRatingsTest_mask] - test_pred_ols))
print("Test_ols MAE  %s " % (test_mae_ols/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

test_mse_ols = np.mean((MusicRatingsTest_mat[MusicRatingsTest_mask] - test_pred_ols)**2)
print("Test_ols RMSE %s " % (np.sqrt(test_mse_ols)/4))

print("Test_ols OSR2 %s " % OSR2(test_mse_ols, baseline_mse))

In [ ]:
dtm = pd.concat([MusicRatingsTrain, MusicRatingsTest], join = 'inner')
MusicRatingsTrain_new = dtm.iloc[0:MusicRatingsTrain.shape[0]]
MusicRatingsTest_new = dtm.iloc[MusicRatingsTest.shape[0]:]

In [ ]:
MusicRatingsTest_new.reset_index(drop=True, inplace=True)
test_pred_rf=rf.predict(MusicRatingsTest.drop(columns = ["songID","rating"]))

blend_test_df['test_pred_rf']=test_pred_rf
blend_test_df

In [ ]:
test_mae_rf = np.mean(np.abs(MusicRatingsTest_mat[MusicRatingsTest_mask] - test_pred_rf))
print("Test_rf MAE  %s " % (test_mae_rf/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

test_mse_rf = np.mean((MusicRatingsTest_mat[MusicRatingsTest_mask] - test_pred_rf)**2)
print("Test_rf RMSE %s " % (np.sqrt(test_mse_rf)/4))

print("Test_rf OSR2 %s " % OSR2(test_mse_rf, baseline_mse))

In [ ]:
blend_test_temp = blend_test_df.rename(columns={"test_pred_cf":"val_pred_cf","test_pred_ols":"val_pred_ols","test_pred_rf":"val_pred_rf" })
test_pred_blended =blending_res.predict(blend_test_temp)
blend_test_df['pred_blended']= test_pred_blended
blend_test_df

In [ ]:
test_mae_blended = np.mean(np.abs(MusicRatingsTest_mat[MusicRatingsTest_mask] - test_pred_blended))
print("Test_blended MAE  %s " % (test_mae_blended/4)) #Note that we normalize MAE and RMSE by the scale of the ratings, which is 5-1=4.

test_mse_blended = np.mean((MusicRatingsTest_mat[MusicRatingsTest_mask] - test_pred_blended)**2)
print("Test_blended RMSE %s " % (np.sqrt(test_mse_blended)/4))

print("Test_blended OSR2 %s " % OSR2(test_mse_blended, baseline_mse))